In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys
import pickle as pk
import time
import pandas as pd

import utils as ut
import time
import math
import copy
from sklearn.preprocessing import normalize
from itertools import chain
from GPyOpt.methods import BayesianOptimization
import scipy
from functools import partial

In [2]:
# constants
# rhino uv nurbs +1 (even numbers required?
u = 8 #u = number of points
v = 5 #v = number of platforms
#f_experiment = '200616'
curr_dir_repo = os.getcwd().split('/src/python')[0]
dataset_dir = curr_dir_repo + '/data/' #+ f_experiment
results_dir = curr_dir_repo + '/results/' #+ f_experiment

fn_attr_ranges = 'ranges_attrib.pkl'

###
# THE NOTATION IS
# W for geometries: the points, either on xyz or polar, for each of the platforms
# X for desing attributes: final desired characteristics, like rain occlusion,
# sun occlusion, surface, etc.

flag_polar = True

fname = os.path.join(dataset_dir, 'data_labels_all_200706.pkl')
list_all_dicts = pk.load(open(fname, 'rb'))

# %%

flag_GH_xyz_polar = 0
flag_out_xyz_polar = 0
if flag_polar:
    flag_GH_xyz_polar = 1
    flag_out_xyz_polar = 1

w_train_red, bottom_top_heights = ut.gh_to_script(list_all_dicts, v, u, flag_GH_xyz_polar=flag_GH_xyz_polar,
                                                  flag_out_xyz_polar=flag_out_xyz_polar)

# %%
# Import and preprocess y_train: design attributes


# Attributes to use
list_attrib = ['occlusion_rain', 'occlusion_sun',
               'outline_lengths_0', 'outline_lengths_1', 'outline_lengths_2', 'outline_lengths_3', 'outline_lengths_4',
               'surface_areas_0', 'surface_areas_1', 'surface_areas_2', 'surface_areas_3', 'surface_areas_4']
x_train = ut.ghlabels_to_script(list_all_dicts, list_attrib, flag_all_df=True)

all_attributes = list(x_train.columns)
print(all_attributes)


num_attr = len(list_attrib) # number of attributes selected for training

#print("Attributes",list_attrib)

x_train_red = np.asarray(x_train[list_attrib])

w_nans, ind_not_nanw = ut.check_rows_nan(w_train_red)
x_nans, ind_not_nanx = ut.check_rows_nan(x_train_red)

if len(np.union1d(w_nans, x_nans)):
    ind_not_nan = np.intersect1d(ind_not_nanw, ind_not_nanx)
    print('Reducing samples from {} to {}'.format(len(w_train_red), len(ind_not_nan)))
    w_train_red = w_train_red[ind_not_nan, :]
    x_train_red = x_train_red[ind_not_nan, :]
else:
    print('No nans! ¯\_(ツ)_/¯')


data_dir_loop = '/Users/pouya/vegetale_bayopt'
str_save = 'It1'
flag_save_files = True
max_time = 3600
output_attrib = ['occlusion_rain', 'occlusion_sun']


['cantilever_max', 'occlusion_sun', 'outline_lengths_0', 'outline_lengths_1', 'outline_lengths_2', 'outline_lengths_3', 'outline_lengths_4', 'surface_area_total', 'buildable', 'surface_areas_0', 'surface_areas_1', 'surface_areas_2', 'surface_areas_3', 'surface_areas_4', 'outline_length_total', 'supports_count', 'outlines_bleps', 'occlusion_rain', 'parcel_intersection', 'supports_problems', 'columns_max_length']
No nans! ¯\_(ツ)_/¯
